In [22]:
from import_hack import *
from methodsnm.mesh_4d import *
from methodsnm.visualize import *
import math
import numpy as np
np.math = math
m=2
mesh = StructuredTesseraktMesh(m,m,m,m)
#DrawMesh2D(mesh)
#print(mesh.points[mesh.elements()[6]])
for elnr, verts in enumerate(mesh.elements()):
    print(elnr,verts)
    trafo = mesh.trafo(elnr)
    print(trafo.map(np.array([0.5,0.5,0.5,0.5])))
    if elnr == 6:
        break
# print(len(mesh.vertices),len(mesh.bndry_vertices),len(mesh.edges),len(mesh.bndry_edges),len(mesh.faces),len(mesh.bndry_faces),len(mesh.volumes),len(mesh.bndry_volumes))

0 [ 0  1  3  4  9 10 12 13 27 28 30 31 36 37 39 40]
[0.25 0.25 0.25 0.25]
1 [27 28 30 31 36 37 39 40 54 55 57 58 63 64 66 67]
[0.25 0.25 0.25 0.75]
2 [ 9 10 12 13 18 19 21 22 36 37 39 40 45 46 48 49]
[0.25 0.25 0.75 0.25]
3 [36 37 39 40 45 46 48 49 63 64 66 67 72 73 75 76]
[0.25 0.25 0.75 0.75]
4 [ 3  4  6  7 12 13 15 16 30 31 33 34 39 40 42 43]
[0.25 0.75 0.25 0.25]
5 [30 31 33 34 39 40 42 43 57 58 60 61 66 67 69 70]
[0.25 0.75 0.25 0.75]
6 [12 13 15 16 21 22 24 25 39 40 42 43 48 49 51 52]
[0.25 0.75 0.75 0.25]


In [23]:
from methodsnm.fes import *
fes = P1_Tesserakt_Space(mesh)
#print(fes.eltype)


In [24]:
from methodsnm.forms import *
from methodsnm.formint import *
from numpy import pi, cos ,sin, exp

#blf = BilinearForm(fes)
#c = GlobalFunction(lambda x: 1, mesh = mesh)
#blf += LaplaceIntegral(c)
#blf += MassIntegral(c)
#blf.assemble()
#
#lf = LinearForm(fes)
#f = GlobalFunction(lambda x: (1+4*pi**2)*sin(pi*x[0])*sin(pi*x[1])*sin(pi*x[2]), mesh = mesh)
#lf += SourceIntegral(f)
#lf.assemble()

In [25]:
#print(blf.matrix, "\n", lf.vector)

In [26]:
#uh = FEFunction(fes)
#left = mesh.filter_bndry_points("min",0)
#right = mesh.filter_bndry_points("max",0)
#bndry = left +right

def list_diff(a, b):
    """Entfernt alle Elemente aus Liste a, die in Liste b enthalten sind."""
    return [x for x in a if x not in b]

from scipy.sparse.linalg import spsolve
from methodsnm.solver import solve_on_freedofs
#freedofs = list_diff(mesh.vertices,mesh.bndry_vertices)
##uh.vector = spsolve(blf.matrix, lf.vector)
#uh.vector = solve_on_freedofs(blf.matrix,lf.vector,freedofs)

In [27]:
from methodsnm.forms import compute_difference_L2

#uex =  GlobalFunction(lambda x: sin(pi*x[0])*sin(pi*x[1])*sin(pi*x[2]), mesh = mesh)
#l2diff = compute_difference_L2(uh, uex, mesh, intorder = 3)

#print("l2diff =", l2diff)

In [28]:
uD = lambda x: cos(pi*x[0])*cos(pi*x[1])*cos(pi*x[2])
#uD = lambda x: 0
top= mesh.filter_bndry_points("max",3)
bottom= mesh.filter_bndry_points("min",3)
#bndry= mesh.bndry_vertices
#bndry = list_diff(mesh.bndry_vertices,top)
bndry = bottom
freedofs = list_diff(mesh.vertices,bndry)
uh2 =FEFunction(fes)
uh2._set(uD, bndry=bndry)

In [29]:
blf2 = BilinearForm(fes)
c = GlobalFunction(lambda x: 1, mesh = mesh)
blf2 += LaplaceIntegral_without_time1(c)
blf2 += TimeIntegral(c)
blf2.assemble()

#uex =  GlobalFunction(lambda x: x[3], mesh = mesh)
#uh2._set(lambda x: x[3], False)

In [30]:

lf2 = LinearForm(fes)
#f2 = GlobalFunction(lambda x: 3*pi**2*sin(pi*x[0])*sin(pi*x[1])*sin(pi*x[2]), mesh = mesh)
#f2 = GlobalFunction(lambda x: 3*pi**2*cos(pi*x[0])*cos(pi*x[1])*cos(pi*x[2]), mesh = mesh)
f3 = GlobalFunction(lambda x: 1, mesh = mesh)
#lf2 += SourceIntegral(f2)
#lf2 += SourceIntegral(f3)
lf2.assemble()
#print(lf2.vector)

In [31]:
#print(blf2.matrix*np.array([0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1]))

In [32]:
lf2.vector -= blf2.matrix * uh2.vector
uh2.vector += solve_on_freedofs(blf2.matrix,lf2.vector,freedofs)
#print(uh2.vector)
uex =  GlobalFunction(lambda x: exp(-3*pi**2*x[3])*cos(pi*x[0])*cos(pi*x[1])*cos(pi*x[2]), mesh = mesh)
#uex =  GlobalFunction(lambda x: x[3] + cos(pi*x[0])*cos(pi*x[1])*cos(pi*x[2]), mesh = mesh)
#uex =  GlobalFunction(lambda x: x[3], mesh = mesh)

#uex =  GlobalFunction(lambda x: x[3], mesh = mesh)
l2diff = compute_difference_L2(uh2, uex, mesh, intorder = 3)

print("l2diff =", l2diff)

l2diff = 0.054395741543702435


In [33]:
print(uh2.vector)

[ 1.00000000e+00  6.12323400e-17 -1.00000000e+00  6.12323400e-17
  3.74939946e-33 -6.12323400e-17 -1.00000000e+00 -6.12323400e-17
  1.00000000e+00  6.12323400e-17  3.74939946e-33 -6.12323400e-17
  3.74939946e-33  2.29584502e-49 -3.74939946e-33 -6.12323400e-17
 -3.74939946e-33  6.12323400e-17 -1.00000000e+00 -6.12323400e-17
  1.00000000e+00 -6.12323400e-17 -3.74939946e-33  6.12323400e-17
  1.00000000e+00  6.12323400e-17 -1.00000000e+00 -3.28519856e-01
  1.24186824e-12  3.28519856e-01  1.24188211e-12 -3.25074954e-17
 -1.24195844e-12  3.28519856e-01 -1.24195497e-12 -3.28519856e-01
 -1.99239413e-13  2.83950843e-17  1.99161698e-13 -1.15222054e-17
  9.35882401e-18  3.08510427e-17  1.99170367e-13  5.51269375e-18
 -1.99236529e-13  3.28519856e-01 -1.24193384e-12 -3.28519856e-01
 -1.24193763e-12 -7.00341299e-19  1.24190120e-12 -3.28519856e-01
  1.24190120e-12  3.28519856e-01  1.76895307e-01 -2.82709103e-13
 -1.76895307e-01 -2.82719511e-13 -1.21756609e-16  2.82705633e-13
 -1.76895307e-01  2.82688